In [6]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')

from lightning_train import DataModule, generate_trainer
from data import *
from model import *

from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *

import pandas as pd
from scipy.sparse import csr_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from functools import partial
import torchmetrics.functional as f
from torchmetrics import Metric
import torchmetrics 

In [2]:
data = an.read_h5ad('../data/retina/retina_T.h5ad')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [3]:
cols = data.var['gene']
ref = cols[5:]

In [4]:
len(cols), len(ref)

(37475, 37470)

In [7]:
module = DataModule(
    datafiles=['../data/retina/retina_T.h5ad'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
    batch_size=16,
    num_workers=0,
    shuffle=True,
    drop_last=True,
    normalize=True,
    deterministic=True,
    currgenes=cols,
    refgenes=ref,
)

module.setup()

model = SIMSClassifier(
    input_dim=len(ref),
    output_dim=module.num_labels,
)

Creating train/val/test DataLoaders...


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Done, continuing to training.
Calculating weights
Initializing network
Initializing explain matrix


In [ ]:
wandb_logger = WandbLogger(
    project=f"Retina Model",
    name='local-retina-model'
)

trainer = pl.Trainer(logger=wandb_logger)
trainer.fit(model, datamodule=module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.001
)


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 2.5 M 
-----------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.915     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Let's do some simple preprocessing for training, and then test our library

In [4]:
# le = LabelEncoder()

# labels = pd.read_csv('../data/retina/retina_labels.tsv', sep='\t')
# labels = labels[labels['CellType'].isin(['retinal rod cell type B', 'retinal rod cell type A'])]
# labels = labels[labels['CellType'] != 'unannotated']
# labels['class_label'] = le.fit_transform(labels['CellType'])

# labels.index.name = 'cell'
# # labels = labels.iloc[0:5000, :]
# labels.to_csv('../data/retina/retina_labels_numeric.csv')

# # label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
# # label_df

# # labels.to_csv('../data/retina/retina_labels_numeric.csv')

# labels = pd.read_csv('../data/retina/retina_labels_numeric.csv')
# labels.loc[:, 'class_label']

In [5]:
# labels = pd.read_csv('../data/retina/raw_labels.tsv', sep='\t')
# corrected = pd.read_csv('../data/retina/retina_labels_numeric.csv')

In [6]:
# from sklearn.model_selection import train_test_split
# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, random_state=42)
# trainsplit

In [7]:
corrected = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
print(corrected.shape)
# current_labels = corrected['CellType']

# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, random_state=42)
# trainsplit

(16446, 8)


In [8]:
module = DataModule(
    datafiles=['../data/retina/retina_T.h5ad'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
    batch_size=16,
    num_workers=0,
    shuffle=True,
    drop_last=True,
    normalize=True,
    deterministic=True,
)

module.setup()

Creating train/val/test DataLoaders...


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Done, continuing to training.
Calculating weights


In [9]:
module.valloader.dataset.labels

array([10,  9,  7, ...,  9,  9,  9])

In [10]:
import pytorch_lightning as pl

lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')
checkpoint = pl.callbacks.ModelCheckpoint(
    dirpath='checkpoints', 
    filename='{epoch}-{weighted_val_accuracy}'
)
# pruner = pl.callbacks.ModelPruning()
progressbar = pl.callbacks.RichProgressBar()

In [11]:
tabnetmodel = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.001,
    },
    metrics={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
)

wandb_logger = WandbLogger(
    project=f"Retina Model",
    name='local-retina-model'
)

early_stop = EarlyStopping(
    monitor="weighted_val_accuracy", 
    min_delta=0.00, 
    patience=3,
    verbose=False, 
    mode="max"
)

trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[early_stop, lr_monitor, checkpoint],
    max_epochs=100,
)

Initializing network
Initializing explain matrix


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [12]:
trainer.fit(tabnetmodel, datamodule=module)

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 2.5 M 
-----------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.916     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Median f1 score is 0.0 for epoch=0
Validation F1-scores are [0.         0.         0.         0.         0.         0.
 0.         0.14285714 0.         0.         0.33333333 0.
 0.        ]


/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:177: RuntimeWarning: invalid value encountered in true_divide
  precision = tp / (tp + fp)
/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:178: RuntimeWarning: invalid value encountered in true_divide
  recall = tp / (tp + fn)
/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:179: RuntimeWarning: invalid value encountered in true_divide
  f1s = 2*(precision * recall) / (precision + recall)
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [13]:
to_test = pd.read_csv('../data/retina/fovea_for_test_labels.tsv', sep='\t')

In [14]:
train_labels = pd.read_csv('../data/retina/retina_labels_numeric.csv')

In [15]:
to_test

,cellId,final_cluster_labels,libraryID,celltype,region,disease_ontology_term_id,disease,sex,tissue_ontology_term_id,tissue,cell_type_ontology_term_id,cell_type
0,AAACCCAGTCGGCACT-1,9,fovea_donor_1,Unknown,fovea,MONDO:0005041,glaucoma (disease),male,UBERON:0001786,fovea centralis,NaN,unknown
1,AAACGAACAGGGACTA-1,15,fovea_donor_1,Glial-F1,fovea,MONDO:0005041,glaucoma (disease),male,UBERON:0001786,fovea centralis,CL:0000125,glial cell
2,AAACGAAGTTCTCCAC-1,3,fovea_donor_1,Cones-Fov,fovea,MONDO:0005041,glaucoma (disease),male,UBERON:0001786,fovea centralis,NaN,foveal cone photoreceptor
3,AAACGAATCATAGCAC-1,5,fovea_donor_1,Bipolar-1,fovea,MONDO:0005041,glaucoma (disease),male,UBERON:0001786,fovea centralis,CL:0000103,bipolar neuron
4,AAACGCTAGGAGGCAG-1,8A,fovea_donor_1,Horizontal,fovea,MONDO:0005041,glaucoma (disease),male,UBERON:0001786,fovea centralis,CL:0000745,retina horizontal cell
...,...,...,...,...,...,...,...,...,...,...,...,...
8212,TTTGACTCATGACTGT-6,5,peripheral_donor_3,Bipolar-1,peripheral,PATO:0000461,normal,female,UBERON:0013682,peripheral region of retina,CL:0000103,bipolar neuron
8213,TTTGACTGTCCGGACT-6,6,peripheral_donor_3,Bipolar-2,peripheral,PATO:0000461,normal,female,UBERON:0013682,peripheral region of retina,CL:0000103,bipolar neuron
8214,TTTGACTTCTGAGTCA-6,2,peripheral_donor_3,Rods-2,peripheral,PATO:0000461,normal,female,UBERON:0013682,peripheral region of retina,CL:0000604,retinal rod cell
8215,TTTGGAGCAGTTTGGT-6,6,peripheral_donor_3,Bipolar-2,peripheral,PATO:0000461,normal,female,UBERON:0013682,peripheral region of retina,CL:0000103,bipolar neuron


In [16]:
train_labels

,cell,index,Expressed Genes,UMI Count,Percent Mitochond.,BroadCellType,CellType,Donor,class_label
0,0,0024369980fd003553cbc9dfe29f7f95,2351,6060.0,4.125413,retinal rod cell,retinal rod cell type A,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,9
1,2,0037f1b36684cb59b84d3585ca55ff69,950,1507.0,0.597213,retinal rod cell,retinal rod cell type B,427c0a62-9baf-42ab-a3a3-f48d10544280,10
2,3,00390952646f52d11a9ab9bba7d6ac51,961,1962.0,7.543323,retinal rod cell,retinal rod cell type A,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,9
3,4,005b3351658380695a5dc46c384d72d7,858,1384.0,0.144509,retinal rod cell,retinal rod cell type B,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,10
4,6,00906832f470fc434a52ac7d678a95bc,532,1054.0,6.451613,retinal rod cell,retinal rod cell type A,427c0a62-9baf-42ab-a3a3-f48d10544280,9
...,...,...,...,...,...,...,...,...,...
16441,19684,ff61c0c282f41e4a37885b05342441da,1494,3320.0,2.289157,retinal rod cell,retinal rod cell type B,427c0a62-9baf-42ab-a3a3-f48d10544280,10
16442,19689,ffa4633bef82949d2c6ac17b3ddf46e9,1813,4129.0,3.996125,retinal rod cell,retinal rod cell type A,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,9
16443,19690,ffa5758b0600f47722fdc755444dfe0c,2692,8244.0,16.460457,retinal cone cell,retinal cone cell,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,7
16444,19691,ffd3fd6119de767f3c3b8c47b2c28bf0,516,896.0,3.794643,retinal rod cell,retinal rod cell type A,427c0a62-9baf-42ab-a3a3-f48d10544280,9


In [17]:
to_test['cell_type'].unique(), train_labels['CellType'].unique()

(array(['unknown', 'glial cell', 'foveal cone photoreceptor',
        'bipolar neuron', 'retina horizontal cell', 'endothelial cell',
        'pericyte cell', 'retinal rod cell', 'retinal ganglion cell',
        'amacrine cell', 'microglial cell',
        'peripheral cone photoreceptor'], dtype=object),
 array(['retinal rod cell type A', 'retinal rod cell type B',
        'retinal bipolar neuron type B', 'retinal rod cell type C',
        'unspecified', 'retinal bipolar neuron type C', 'Muller cell',
        'retinal cone cell', 'retinal bipolar neuron type A',
        'amacrine cell', 'retinal bipolar neuron type D',
        'retinal ganglion cell', 'microglial cell'], dtype=object))

In [18]:
# map to_test --> training labels for testing prediction on other datasets

# mapping = {
#     'glial cell': 'microglial',
#     'foveal cone photoreceptor': 
#     'bipolar neuron': 
# }

In [ ]:
model = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.001,
    },
    metrics={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
)

Initializing network
Initializing explain matrix


In [ ]:
model = TabNetLightning.load_from_checkpoint(
    '../checkpoints/checkpoint-80-desc-retina.ckpt',
    input_dim=37475,
    output_dim=13,
    n_d=32,
    n_a=32,
    n_steps=10,
)

In [ ]:
model

In [ ]:
datafile = '../data/retina/fovea_for_test_T.h5ad'
data = an.read_h5ad('../data/retina/fovea_for_test_T.h5ad')

dataset = AnnDataMatrix(
    data=data.X,
    labels=
)

In [ ]:
!ls ../data/retina

In [ ]:
labels = pd.read_csv('../data/retina/fovea_for_test_labels.tsv', sep='\t')

In [ ]:
labels